In [ ]:
# Создание и преобразование столбцов с помощью функций
# .apply() - метод
# функция, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу 

In [19]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
# display(melb_data.head())
melb_df = melb_data.copy()
# display(melb_df.head())
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
#melb_df.head()
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea']/total_rooms
#display(melb_df['MeanRoomsSquare'])
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])
price_square = melb_df['Price']**2
#display(price_square)

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# print(melb_df['Date'])

year_sold = melb_df['Date'].dt.year
#print('год продажи', year_sold)
#print('min yearsold', year_sold.min())
#print('max yearsold', year_sold.max())
#print('mode of sold', year_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop(['YearBuilt'], axis=1)
#display(melb_df.head())

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
#print(weekend_count, 'объектов недвижимости было продано в выходные')


In [20]:
# В наших данных есть столбец с адресами объектов недвижимости
# check количество уникальных значений в столбце с помощью метода nunique()
print('количество уникальных значений в столбце', melb_df['Address'].nunique())

количество уникальных значений в столбце 13378


In [21]:
# давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [22]:
import pandas as pd
# Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E'] # Создаём список географических пометок exclude_list.
    address_list = address.split(' ') # Метод split() разбивает строку на слова по пробелу, получаем список слов в строке 
    street_type = address_list[-1] # Обрезаем список, оставляя в нём только последний элемент, потенциальный подтип улицы
    if street_type in exclude_list: # Делаем проверку на то, что полученный подтип является географической пометкой at exclude_list
        street_type = address_list[-2] # Если переменная street_type является географической пометкой change на второй элемент с конца 
    return street_type
# Теперь применим эту функцию к столбцу c адресом 
# передадим функцию get_street_type в аргумент метода столбца apply()
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)
# функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [23]:
# мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось
print(street_types.nunique())

56


In [24]:
# Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:
display(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

In [31]:
# метод уменьшения количества уникальных категорий
# выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).
# Для этого к результату метода value_counts применим метод nlargest()
# возвращает n наибольших значений из Series
# Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. 
# Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:

popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


In [32]:
# Теперь, когда у нас есть список наиболее популярных подтипов улиц, 
# введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, 
# и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'
# а результат определим в новый столбец таблицы StreetType

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'others')
display(melb_df['StreetType'])

0            St
1            St
2            St
3        others
4            St
          ...  
13575        Cr
13576        Dr
13577        St
13578        St
13579        St
Name: StreetType, Length: 13580, dtype: object

In [33]:
# Посмотрим на результирующее число уникальных подтипов:
print(melb_df['StreetType'].nunique())

11


In [38]:
# Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:
import pandas as pd
#melb_df = melb_df.drop('Address', axis=1)
display(melb_df.head())

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Lattitude,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType
0,Abbotsford,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067,2,...,-37.7996,144.9984,Northern Metropolitan,4019,25.20,-0.231707,12,46,5,St
1,Abbotsford,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067,2,...,-37.8079,144.9934,Northern Metropolitan,4019,15.80,-0.327660,2,116,3,St
2,Abbotsford,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,3067,3,...,-37.8093,144.9944,Northern Metropolitan,4019,18.75,0.056338,3,117,5,St
3,Abbotsford,3,h,850000.0,PI,Biggin,2017-03-04,2.5,3067,3,...,-37.7969,144.9969,Northern Metropolitan,4019,15.75,0.145455,3,47,5,others
4,Abbotsford,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,3067,3,...,-37.8072,144.9941,Northern Metropolitan,4019,17.75,0.083969,6,2,5,St


In [ ]:
# Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё
